In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Code xử lý dữ liệu tháng 12.2021

Phần 1: xử lý cụm từ và trích xuất được cặp cụm từ trigram vi-en dịch lẫn nhau.
Bước xử lý cho cụm từ vi-en
Bước 1: xử lý thô phrase-table vi-en
Bước 2: xử lý trích xuất cụm từ vi-en trong đó cụm vi là trigram.
Bước 3: xử lý trích xuất cụm từ vi-en trong đó có xác suất dịch là lớn nhất file bangcumtu-3gram-vi-en.txt
Thực hiện tương tự cho cụm từ en-vi ta được file bangcumtu-3gram-en-vi.txt

Cuối cùng ta so sánh hai file bangcumtu-3gram-vi-en.txt và bangcumtu-3gram-en-vi.txt nếu cụm từ vi-en có ở cả 2 file trên thì nó chính là cụm từ cần tìm cuối cùng ( cụm từ dịch lẫn nhau)

#############################
Xử lý bước 2 để loại bỏ bớt những cụm từ tiếng Việt bị lặp từ mà ý nghĩa không thay đổi trong cụm En, cụ thể nếu cụm dịch En là giống nhau thì ta chọn cụm nguồn Vi có chiều dài là dài nhất.


In [ ]:
# Cài đặt Tool Tokenize
#! pip install underthesea
#! pip install mosestokenizer

In [8]:
# Cấu hình word tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize #tool tokenize tiếng anh
#from underthesea import word_tokenize # tool tokenize tiếng việt
#from mosestokenizer import *
import re
#import gzip

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
%cd '/content/drive/My Drive/'

/content/drive/My Drive


In [ ]:
# Khai báo đường dẫn file phrase-table, có 2 loại phrase-table là dịch tiếng Việt - tiếng Anh và ngược lại
# phrase-table vi-en
SOURCE_FILE = 'data/bangcumtu/phrase-table-trigram-vi-en'
TARGET_FILE = 'data/bangcumtu/phrase-table-trigram-vi-en-clean'

'''
# phrase-table en-vi
SOURCE_FILE = 'data/train/phrase-table-en-vi'
TARGET_FILE = 'data/train/phrase-table-en-vi-clean.txt'
'''


"\n# phrase-table en-vi\nSOURCE_FILE = 'data/train/phrase-table-en-vi'\nTARGET_FILE = 'data/train/phrase-table-en-vi-clean.txt'\n"

In [ ]:
# xóa những cụm từ chứa những ký tự đặc biệt {.:;,$&*?/\|!+-]+}
# xử lý chung cho cụm vi và cụm en

import re
with open(SOURCE_FILE,'r', encoding="utf8") as file_1, open(TARGET_FILE,'a', encoding="utf8") as file_2:
    for lines in file_1:
        line = lines.rstrip().split('|||')
        check1 = re.findall(r'[.:;,$%&*?/\!+-]+', line[1])
        #check2 = re.findall('\d', line[0])# kiểm tra cụm vn toàn số thì loại bỏ
        check2 = re.findall('\d', line[1])# kiểm tra cụm vn nếu chứa toàn số thì loại bỏ
        check3 = re.findall(r'[.:;,$%&*?/\!+-]+', line[0])
        #print(l_vi)
        if not check1 and not check2 and not check3: 
            file_2.write(lines)
#print(random.sample(tmp_text,20))

In [ ]:
#Count number lines
with open(SOURCE_FILE, 'r', encoding='utf8') as file1:
    Counter1 = 0
  
# Reading from file 
    Content = file1.read() 
    CoList = Content.split("\n") 
  
    for i in CoList: 
        if i: 
            Counter1 += 1
file1.close()
         

with open(TARGET_FILE, 'r', encoding='utf8') as file2:
    Counter2 = 0
  
# Reading from file 
    Content = file2.read() 
    CoList = Content.split("\n") 
  
    for i in CoList: 
        if i: 
            Counter2 += 1
file2.close()
# Print results
print("This is the number of lines in the SOURCE_FILE: ",Counter1)
print("This is the number of lines in the TARGET_FILE: ",Counter2) 

This is the number of lines in the SOURCE_FILE:  6418963
This is the number of lines in the TARGET_FILE:  3518608


In [4]:
# Khai báo function Xử lý file phrase-table vi-en
#Xử lý rút ra được các cụm từ vi-en mà trong đó cụm vi là trigram
# xóa các cụm từ khuyết cụm en, bắt đầu bằng dấu "!,.:?\""

source_text = []
combine_text_vi_en = []
combine_raw = []
#vn_tokenize = MosesTokenizer('vi') # tokenize tiếng Việt
#en_tokenize = MosesTokenizer('en') # tokenize tiếng Anh
# Khởi tạo các hàm xử lý chuỗi
def InsertSpace(token):
    result = ''
    
    for text in token:
        result += text + ' '
    
    return result.rstrip()
'''
def normalizeString(s):
    s = html.unescape(s)
    # Seperate words and marks by adding spaces between them
    #marks = '[.!?,-${}()]'
    #r = "(["+"\\".join(marks)+"])"
    #s = re.sub(r, r" \1 ", s)
    # replace continuous spaces with a single space
    s = re.sub(r"\s+", r" ", s).strip()
    return s
'''

# code xử lý cụm tiếng việt - tiếng anh
def FilterPairs_vi_en (l):
    # tokenzie word by nltk
    source_tokenize = word_tokenize(l[0])
    target_tokenize = word_tokenize(l[1])
    # tokenize with mosestokenizer
    #source_tokenize = vn_tokenize(l[0])
    #target_tokenize = en_tokenize(l[1])
    # Thêm space vào các token từ
    source_raw = InsertSpace(source_tokenize)
    target_raw = InsertSpace(target_tokenize)
         
    # Kiểm tra độ dài của cụm từ vi thỏa điều kiện
    if len(source_tokenize) == 3  and len(target_tokenize) >= 2 :
        combine_raw = source_raw + '|||' + target_raw + '|||' + l[2] #l[2] là các thông số xác suất dịch của bảng cụm từ
        #combine_text = source_raw + '\t' + target_raw
        combine_text_vi_en.append(combine_raw)
        #print(combine_text[0])
    #l = combine_text

    return combine_text_vi_en

# Xử lý file nguồn

print('Finish')


Finish


In [ ]:
# Khai báo function Xử lý file phrase-table en-vi
#Xử lý rút ra được các cụm từ vi-en mà trong đó cụm vi là trigram
# xóa các cụm từ khuyết cụm en, bắt đầu bằng dấu "!,.:?\""

source_text = []
combine_text = []
combine_raw = []
vn_tokenize = MosesTokenizer('vi') # tokenize tiếng Việt
en_tokenize = MosesTokenizer('en') # tokenize tiếng Anh
# Khởi tạo các hàm xử lý chuỗi
def InsertSpace(token):
    result = ''
    
    for text in token:
        result += text + ' '
    
    return result.rstrip()
'''
def normalizeString(s):
    s = html.unescape(s)
    # Seperate words and marks by adding spaces between them
    #marks = '[.!?,-${}()]'
    #r = "(["+"\\".join(marks)+"])"
    #s = re.sub(r, r" \1 ", s)
    # replace continuous spaces with a single space
    s = re.sub(r"\s+", r" ", s).strip()
    return s
'''
# Code xử lý chuỗi tiếng Anh - tiếng Việt

def FilterPairs (l):
    source_tokenize = en_tokenize(l[0]) # tokenize en
    target_tokenize = vn_tokenize(l[1]) # tokenize vi
    source_raw = InsertSpace(source_tokenize)
    target_raw = InsertSpace(target_tokenize)
    #combine_text = source_raw.rstrip() + '|||' + target_raw.rstrip()
    

    
    # Kiểm tra độ dài của cụm từ vi thỏa điều kiện
    if len(source_tokenize) >= 2  and len(target_tokenize) == 3 :
        # Đảo cụm từ vi ra trước cụm từ en
        combine_raw = target_raw + '|||' + source_raw + '|||' + l[2]# đảo vị trí của cụm từ vi lên trước cụm từ en nhằm mục tiêu xác định dịch lẫn nhau ở bước tiếp theo
        #combine_text = source_raw + '\t' + target_raw
        combine_text.append(combine_raw)
        #print(combine_text[0])
    #l = combine_text

    return combine_text

# Xử lý file nguồn

print('Finish')


In [ ]:
# Code xử lý cụm từ để tạo trigram phrase-table vi-en
PHRASE_FILE_VI_EN = 'data/bangcumtu/phrase-table-vi-en'
PHRASE_TARGET_VI_EN = 'data/bangcumtu/phrase-table-trigram-vi-en'
import random
import numpy
#result_text = []
with open(PHRASE_FILE_VI_EN,'r', encoding="utf8") as file1:
    for line in file1:
        lines = line.strip().split('|||')
        new_lines = FilterPairs_vi_en(lines) #    

# Lưu các cụm từ 3 gram vào file
with open(PHRASE_TARGET_VI_EN,'w', encoding="utf8") as file2:
    for item in combine_text_vi_en:
        #print('%s\t' % item)
        file2.write("%s\n" % item)

print('Finished')

Finished


In [ ]:
# Code xử lý cụm từ để tạo trigram phrase-table en-vi
PHRASE_FILE = 'data/train/phrase-table-en-vi-clean.txt'
PHRASE_TARGET = 'data/train/phrase-table-en-vi-3gram.txt'
import random
import numpy
#result_text = []
with open(PHRASE_FILE,'r', encoding="utf8") as file1:
    for line in file1:
        lines = line.strip().split('|||')
        new_lines = FilterPairs(lines) #    

# Lưu các cụm từ 3 gram vào file
with open(PHRASE_TARGET,'w', encoding="utf8") as file2:
    for item in combine_text:
        #print('%s\t' % item)
        file2.write("%s\n" % item)

print('Finished')

In [ ]:
# Gộp xác suất dịch phrase-table
# code này đang chạy trên spyder
# Khai báo phrase-table trigram vi-en
SOURCE_FILE_VI_EN = 'data/bangcumtu/phrase-table-trigram-vi-en-clean'
TARGET_FILE_VI_EN = 'data/bangcumtu/phrase-table-trigram-vi-en-clean-gop-p'

# Khai báo phrase-table trigram en-vi
#SOURCE_FILE = 'data/train/phrase-3gram-en-vi.txt'
#TARGET_FILE = 'data/train/phrase-3gram-en-vi-gop-p.txt'

# cụm từ dạng vi|||en||| 4 giá trí xác suất dịch : chiếc máy tính|||front of a single computer||| 0.2 8.90453e-11 1 0.000276122

# xử lý gộp xác suất cho bảng cụm từ vi-en
with open(SOURCE_FILE_VI_EN,'r' , encoding="utf-8") as file_1, open(TARGET_FILE_VI_EN,'w' , encoding="utf-8") as file_2:
    for lines in file_1:
        l1 = lines.rstrip().split('|||')
        p3_l = ''.join(l1[2])
        #print(p3_l)
        p3_l1 = p3_l.rstrip().split(' ')
        #print('xác suất: ',p3_l1[0],'-',p3_l1[1],'-',p3_l1[2], '-',p3_l1[3], '-',p3_l1[4])
        num = float(p3_l1[1]) + float(p3_l1[2]) 
        l = l1[0] + '|||' + l1[1] + '|||' + str(num) 
        
        file_2.write('%s\n'%l)

'''
#Xử lý gộp xác suất cho bảng cụm từ en-vi
with open(SOURCE_FILE,'r' , encoding="utf-8") as file_3, open(TARGET_FILE,'w' , encoding="utf-8") as file_4:
    for lines in file_3:
        l1 = lines.rstrip().split('|||')
        p3_l = ''.join(l1[2])
        #print(p3_l)
        p3_l1 = p3_l.rstrip().split(' ')
        #print('xác suất: ',p3_l1[0],'-',p3_l1[1],'-',p3_l1[2], '-',p3_l1[3], '-',p3_l1[4])
        num = float(p3_l1[1]) + float(p3_l1[2]) 
        l = l1[0] + '|||' + l1[1] + '|||' + str(num) 
        
        file_4.write('%s\n'%l)
'''
print('Finished')

Finished


In [ ]:
# Chọn cụm từ có xác suất gộp lớn nhất cho bảng cụm từ vi-en
import operator
#from operator import itemgetter
# Lộc cụm từ vn-en với điều kiện xác suất gộp là lớn nhất
# sử dụng định dạng từ điển (dictionary) để lộc cụm en
#khai báo đường dẫn file

# Tạo từ điển để tìm cụm en dựa trên điều kiện id là cụm vn '|||' xác suất
SOURCE_FILE = 'data/bangcumtu/phrase-table-trigram-vi-en-clean-gop-p'
# File chứa cụm vn và xác suất gộp lớn nhất

# Kết quả
TARGET = 'data/bangcumtu/bangcumtu-3gram-vi-en.txt'
bangCumTu = {}
source_text = []
expectedBangCumTu = {}
with open(SOURCE_FILE,'r', encoding='utf-8') as f1:
    for line in f1:
        splitArray = line.split('|||')
        tupleValue = (splitArray[1], splitArray[2])
        bangCumTu[splitArray[0]] = tupleValue
       
for key, value in bangCumTu.items():
    if max(value, key = operator.itemgetter(1)):
        
        expectedBangCumTu[key] = value
    
 
#print('------------Finished-----------')       
'''
# ghi kết quả
'''
with open(TARGET, 'w', encoding='utf-8') as f2:
    for id in expectedBangCumTu:
        #Ghi output_dict vào file kết quả
        f2.write('{0}|||{1}\n'.format(id, expectedBangCumTu[id][0]))

print ('--------Finished---------')

--------Finished---------


In [ ]:
# Chọn cụm từ có xác suất gộp lớn nhất cho bảng cụm từ en-vi
import operator
#from operator import itemgetter
# Lộc cụm từ vn-en với điều kiện xác suất gộp là lớn nhất
# sử dụng định dạng từ điển (dictionary) để lộc cụm en
#khai báo đường dẫn file

# Tạo từ điển để tìm cụm en dựa trên điều kiện id là cụm vn '\t' xác suất
SOURCE_FILE = 'data/train/phrase-3gram-en-vi-gop-p.txt'
# File chứa cụm vn và xác suất gộp lớn nhất
#SOURCE = 'D:\data_phrase_table/thang_11/test-3-en-gop.txt'
# Kết quả
TARGET = 'data/train/bangcumtu-3gram-en-vi.txt'
bangCumTu = {}
source_text = []
expectedBangCumTu = {}
with open(SOURCE_FILE,'r', encoding='utf-8') as f1:
    for line in f1:
        splitArray = line.split('|||')
        tupleValue = (splitArray[1], splitArray[2])
        bangCumTu[splitArray[0]] = tupleValue
       
for key, value in bangCumTu.items():
    if max(value, key = operator.itemgetter(1)):
        
        expectedBangCumTu[key] = value
    
 
#print('------------Finished-----------')       
'''
# ghi kết quả
'''
with open(TARGET, 'w', encoding='utf-8') as f2:
    for id in expectedBangCumTu:
        #Ghi output_dict vào file kết quả
        f2.write('{0}|||{1}\n'.format(id, expectedBangCumTu[id][0]))

print ('--------Finished---------')

In [ ]:
# Sắp xếp lại file cụm từ theo alphabet
S_FILE = 'bangcumtu-3gram-en-vn.txt'
T_FILE = 'data/train/bangcumtu-3gram-en-vn-final.txt'
with open(S_FILE, 'r', encoding='utf8') as f1:
  with open(T_FILE, 'w', encoding='utf8') as f2:
      data=f1.readlines()
      data.sort()
      for i in range(len(data)):
          f2.write(f'{data[i]}')

In [ ]:
# so sanh cum vi-en va en-vi de tim ra cum dich lan nhau
# BỎ NHỮNG LINE KHÔNG GIỐNG NHAU Ở 2 FILE từ đó suy ra được những line giống nhau ở cả hai phrase-table chính là cụm dịch lẫn nhau
from time import time
import random
start = time()
# Đường dẫn file
PHRASE_VI_EN = 'data/bangcumtu-3gram-vi-en.txt'
PHRASE_EN_VI = 'data/bangcumtu-3gram-en-vi.txt'
PHRASE_VI_EN_MUL = 'data/bangcumtu-3gram-vi-en-mul.txt'

with open(PHRASE_VI_EN, 'r', encoding="utf8") as file1:
    with open(PHRASE_EN_VI, 'r', encoding="utf8") as file2:
        same = set(file1).intersection(file2)
  
print("Common Lines in Both Files")
with open(PHRASE_VI_EN_MUL,'w', encoding="utf8") as file3:  
  for line in same:
    file3.write("%s" % line)

file1.close()
file2.close()
file3.close()
print('Finished at:')
print('Time: ',time() - start)

Common Lines in Both Files
Finished at:
Time:  3.295499324798584


Phần 2: Bước xử lý tìm kiếm và thay thế cụm vi thành cụm en trong tập câu song ngữ và lấy mẫu (sampling)

Tìm cụm dịch vi-en trong cặp câu song ngữ của tập train data, tạo thêm data nếu thỏa điều kiện k ( k là số câu song ngữ chứa cặp phrase-table vi-en)
Đoạn code này vì phải xử lý hơn 400k cặp cụm từ vi-en, nên phải chia nhỏ mỗi bước chỉ chạy 100k cặp cụm từ, và chạy code trên Spyder (trên windows 10)

In [ ]:
# code xử lý này được chạy trên Spyder, mỗi lần duyệt qua 100k cặp cụm từ vi-en
#Khai báo hàm tìm kiếm và thay thế cụm vi-en
def search_string_in_file(file_name, vn_1, en_1):
#Search for the given string in file and return lines containing that string,
#along with line numbers
    #line_number = 0
    #list_of_samples = []
    list_of_results = []
    # Open the file in read only mode
    with open(file_name, 'r', encoding="utf-8") as read_obj:
      # Read all lines in the file one by one
      for line in read_obj:
        # For each line, check if line contains the string
          #line_number += 1 # index line numbers
          line_vn, line_en = line.lower().strip().split('|||')
          # kiểm tra điều kiện cụm vi trong câu train tiếng Việt và cụm en trong câu train tiếng Anh
          if vn_1.lower() in line_vn  and en_1.lower() in line_en :
                  # Thay cụm vi = cụm en trong câu tiếng Việt
                  line_raw = line_vn.replace(vn_1.lower(), en_1) +'|||' + line_en
                  
                  list_of_results.append(line_raw.rstrip())
                  # Sampling 20 lines and write to file data
                  #list_of_samples = random.sample(list_of_results)
                  #print(line_raw)
    read_obj.close()
    return list_of_results
#!/usr/bin/env python
start = time.time()
print("Begin")
print(time.ctime())
# Khai bao duong dan file
# Khai báo data train cặp câu vi-en
DATA_TRAIN = 'train-clean-vi-en.txt'
# File chứa cụm từ trigram vi-en 
# File cum tư vn-en có xác suất dịch lớn nhất chứa 450k cặp
PHRASE_FILE = '/data_phrase_table/thang12/bangcumtu-3gram-vn-en.txt'
# File chứa cặp câu vi-en thỏa điều kiện và thực hiện sampling
AUG_FILE = '/data_phrase_table/thang12/sampling-4gram-k5-250k-280k.txt'
# Lưu lại bảng cụm từ vi-en thỏa điều kiện
PHRASE_AUG = '/data_phrase_table/thang12/cum-tu-4gram-thoa-dk-k5-250k-280k.txt'

# Xử lý tìm kiếm cụm từ vi-en

with open(PHRASE_FILE,'r' , encoding="utf-8") as file_1:
    lines = file_1.readlines()
    list_phrase_pairs = []
    list_of_samples = [] 
    
    for line_1 in lines[:100000]: # mỗi lần chạy 100k cụm từ
         vi_first_comb, en_first_comb = line_1.lower().split('|||')
         vi_first_comb, en_first_comb = vi_first_comb.strip(), en_first_comb.strip() # loại dấu cách 
         vi_first_comb , en_first_comb = vi_first_comb +' ', en_first_comb + ' '
         #vi_str1, en_str1 = line_1.strip().split('\t')
         #vi_str1, en_str1 = vi_str1.strip(), en_str1.strip()
         list_results = search_string_in_file(DATA_TRAIN, vi_first_comb, en_first_comb)
         
       
         # Sampling với hệ số k là số câu thỏa điều kiện
         # với k = 5 tức là số cặp câu thỏa điều kiện chứa cụm từ vi-en là lớn hơn 5 câu
         if len(list_results) >= 5 :
             #Sampling results
             results = random.sample(list_results, 5)
                        
             with open(AUG_FILE, 'a', encoding='utf-8') as file_2:
                 
                 for elem in results:
                     file_2.write('%s\n' % elem)
            
             # Write cụm từ
             with open(PHRASE_AUG, 'a', encoding='utf-8') as file_3:
                 
                 file_3.write('%s' % line_1)
                     
                     
        
file_2.close()
file_1.close()
#print('Total Matched lines : ', num_list)

print('Finished')
print(time.ctime())
end = time.time()
print(end - start)# Tìm và thay thế cụm từ trong file train data
             

Gộp file data tăng cường sau khi thay thế cụm vi-en vào trong file data train ban đầu

In [ ]:
import glob
# ghép all file.txt trong thư mục AUG_Sample_10 thành file train data mới
#Copy toàn bộ file data tìm được ở bước 2 và data train ban đầu vào folder /content/drive/My Drive/data/AUG_Sample_5
%cd '/content/drive/My Drive/data/AUG_Sample_5'
read_files = glob.glob("/content/drive/My Drive/data/AUG_Sample_5/*.txt")

with open("result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

Phần 3: tạo data train cho fairseq

In [ ]:
source_file = "/content/drive/My Drive/data/AUG_Sample_5/result.txt"
train_vi = "/content/drive/My Drive/NMT2/data/ntrain.vi"
train_en = "/content/drive/My Drive/NMT2/data/ntrain.en"

list_vi = []
list_en = []

with open(source_file,'r', encoding="utf8") as file:
    for line in file:   
        l = line.strip().split("|||")
        list_vi.append(l[0])
        list_en.append(l[1])
# write to file
with open(train_vi, 'w', encoding='utf8') as file:
    for item in list_vi:
        file.write("%s\n" % item)
with open(train_en, 'w', encoding='utf8') as file:
    for item in list_en:
        file.write("%s\n" % item)

Xử lý bước 2 để tạo cụm dịch 1-1 VI-EN

In [12]:
from nltk import tokenize
# Tạo từ điển để tìm cụm en dựa trên điều kiện id là cụm vn '\t' xác suất
SOURCE = 'data/bangcumtu/bangcumtu-3gram-vi-en.txt'
# File chứa cụm vn và xác suất gộp lớn nhất
#SOURCE = 'D:\data_phrase_table/thang_11/test-3-en-gop.txt'
# Kết quả
TARGET = 'data/bangcumtu/bangcumtu-3gram-vi-en-can-tim.txt'
bangCumTu = {}
source_text = []

with open(SOURCE,'r', encoding='utf-8') as f1:
    for line in f1:
      l = line.replace('_',' ')
      new_l = l.rstrip().split('|||')
      id = ''.join(new_l[1]) # cụm En
      value = ''.join(new_l[0]) # cụm Vi
      if id not in bangCumTu or len(word_tokenize(value))>len(word_tokenize(bangCumTu[id][0])):
          bangCumTu[id] = value
    
 
#print('------------Finished-----------')       
'''
# ghi kết quả
'''
with open(TARGET, 'w', encoding='utf-8') as f2:
    for id in bangCumTu:
        #Ghi output_dict vào file kết quả
        f2.write('{0}|||{1}\n'.format(bangCumTu[id], id))

print('--------Finished---------')

--------Finished---------


In [13]:
# Sắp xếp lại file cụm từ theo alphabet
S_FILE = 'data/bangcumtu/bangcumtu-3gram-vi-en-can-tim.txt'
T_FILE = 'data/bangcumtu/bangcumtu-3gram-vi-en-can-tim-sort.txt'
with open(S_FILE, 'r', encoding='utf8') as f1:
  with open(T_FILE, 'w', encoding='utf8') as f2:
      data=f1.readlines()
      data.sort()
      for i in range(len(data)):
          f2.write(f'{data[i]}')